In [1]:
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np

In [2]:
iris = load_iris()

In [3]:
X = iris.data
y = iris.target

In [4]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [5]:
X.shape

(150, 4)

In [6]:
onehot = OneHotEncoder(categorical_features=[0])
y = onehot.fit_transform(y.reshape(-1,1))
y = y.toarray()

/home/gustavo/Envs/machine/lib/python3.5/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/gustavo/Envs/machine/lib/python3.5/site-packages/sklearn/preprocessing/_encoders.py:390: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [8]:
neuronios_input = X.shape[1]
neuronios_output = y.shape[1]
neuronios_oculta =  np.ceil((X.shape[1]+y.shape[1])/2)
neuronios_oculta = np.int32(neuronios_oculta)

In [9]:
pesos = {
    'oculta':tf.Variable(
        tf.random_normal([neuronios_input, neuronios_oculta]), dtype=tf.float32),
    'saida': tf.Variable(
        tf.random_normal([neuronios_oculta, neuronios_output]), dtype=tf.float32) 
}

bias = {
    'oculta':tf.Variable(
        tf.random_normal([neuronios_oculta])),
    'saida': tf.Variable(
        tf.random_normal([neuronios_output])) 
}

In [10]:
 # Qualquer numero de observações, mas Tem o numero de inputs definido antes
xph = tf.placeholder(dtype=tf.float32, shape=[None, neuronios_input])

# qualquer numero de observações, mas tem 3 saídas por causa do OnehotEncoder
# Essa rede tem 3 saídas. São equivalentes as probabilidades de cada classe
yph = tf.placeholder(dtype=tf.float32, shape=[None, neuronios_output])

In [11]:
def multilayer_perceptron(inputs, weights, bias):
    # multiplicacao dos pesos pelas features
    valores_camada_oculta = tf.matmul(inputs, weights['oculta'])
    
    #adição do bias
    valores_camada_oculta = tf.add(valores_camada_oculta, bias['oculta'])
    
    #Função de ativacao
    valores_camada_oculta = tf.nn.relu(valores_camada_oculta)
    
    #mesmo processo anterior.
    valores_camada_saida = tf.matmul(valores_camada_oculta, weights['saida'])
    valores_camada_saida = tf.add(valores_camada_saida, bias['saida'])
#     valores_camada_saida = tf.nn.softmax(valores_camada_saida)
    
    return valores_camada_saida

In [12]:
modelo = multilayer_perceptron(xph, pesos, bias)

In [15]:
# Definição do erro

erro  = tf.nn.softmax_cross_entropy_with_logits_v2(logits=modelo, labels=yph)
erro = tf.reduce_mean(erro)
otimizador = tf.train.AdamOptimizer().minimize(erro)


In [18]:
BATCH_SIZE = 8
TOTAL_ITERACOES = int(len(X_train)/BATCH_SIZE)
EPOCAS = 3000


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoca in range(EPOCAS):
        erro_medio = 0.0
        
        for i in range(TOTAL_ITERACOES):
            x = X_train[BATCH_SIZE*i:BATCH_SIZE*(i+1)]
            y = y_train[BATCH_SIZE*i:BATCH_SIZE*(i+1)]
            _, custo = sess.run([otimizador, erro], feed_dict={xph:x, yph:y})
            erro_medio += custo/TOTAL_ITERACOES
        if epoca % 500 == 0:
            print("Epoca: ", epoca, " erro: ", erro_medio)
    pesos_finais, bias_final = sess.run([pesos, bias])

Epoca:  0  erro:  4.467739650181362
Epoca:  500  erro:  0.0538054559313293
Epoca:  1000  erro:  0.04613901050886073
Epoca:  1500  erro:  0.04559081233102395
Epoca:  2000  erro:  0.04547841979261388
Epoca:  2500  erro:  0.045447924634832555


In [19]:
print(pesos_finais, bias_final)

{'saida': array([[-8.23778   ,  0.5797728 ,  1.2948929 ],
       [-1.4650621 , -0.2793843 ,  1.5718274 ],
       [ 1.2274582 , -5.1230264 ,  0.732163  ],
       [ 0.17962809,  2.123423  , -3.2874162 ]], dtype=float32), 'oculta': array([[-0.60346025, -0.09025217, -1.5589296 , -0.10256448],
       [-1.3471633 ,  0.18156283,  1.8265883 ,  0.5187789 ],
       [ 3.2531116 , -0.48570377, -2.780385  , -1.6049972 ],
       [ 0.47758478,  2.951703  , -0.8299941 , -1.1728556 ]],
      dtype=float32)} {'saida': array([-4.1828356,  1.2299156,  0.9652812], dtype=float32), 'oculta': array([0.3916647 , 0.38370296, 0.01379086, 2.9992807 ], dtype=float32)}


In [27]:
# previsão

previsao_teste = multilayer_perceptron(xph, pesos_finais, bias_final)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    prev = sess.run(previsao_teste, feed_dict={xph: X_test, yph:y_test})
    #Aplicação da função de probabilidde
    prev = sess.run(tf.nn.softmax(prev))
    
    #Pegando qual coluna tem o valor mais alto, pois, o indice dessa coluna
    # indica qual é a categoria da previsão. (Lembre-se do OneHotEncoder definido antes)
    prev = sess.run(tf.arg_max(prev, 1)) 

In [28]:
prev

array([2, 2, 1, 1, 0, 1, 2, 1, 2, 2, 2, 2, 0, 0, 0, 0, 0, 2, 0, 2, 1, 1,
       1, 2, 2, 1, 1, 1, 2, 0, 1, 0, 0, 2, 1, 0, 2, 2])